# IBX Land Value Uplift and Value Capture Pipeline (MapPLUTO + DOF Market Land Values)

This notebook implements a modular geospatial and fiscal analysis pipeline to estimate how the proposed **Interborough Express (IBX)** could increase **land values** and **property tax revenue** in New York City, using:

- **Parcel geometries and assessed land values** from NYC Department of City Planning’s **MapPLUTO** feature layer.
- **Current market land values** from NYC Department of Finance’s parcel-level **assessment CSV**.
- **Proposed IBX station locations** from `data/stations_ibx.geojson`, which provides the coordinates of each proposed station along the alignment.

The pipeline is loosely modeled on the value-capture framework used by Gupta, Van Nieuwerburgh, and Kontokosta in *“Take the Q Train: Value Capture of Public Infrastructure Projects”* (NBER Working Paper 26789), adapted to a simpler, transparent, parcel-based implementation.

**Key data linkages and fields:**

- **Geometry & parcel IDs:** MapPLUTO 25v3  
  - CRS: `EPSG:2263` (NAD83 / New York Long Island, US feet).  
  - IDs: `BoroCode`, `Block`, `Lot`, `BBL`.  
  - Assessed land: `AssessLand`.  
  - Ownership & use: `OwnerType` (type of ownership) and `LandUse` (01–11 land-use categories).

- **Valuations:** DOF assessment CSV  
  - IDs: `BORO`, `BLOCK`, `LOT` → constructed **BBL**.  
  - **Current market land value:** `CURMKTLAND`.  
  - We **do not use** DOF assessed-land fields; instead we rely on MapPLUTO’s `AssessLand` (which itself is sourced from DOF but packaged in PLUTO).

- **IBX alignment:**  
  - Station locations and order come from `data/stations_ibx.geojson`.  
  - The notebook builds a LineString alignment by connecting stations in order of a `sequence` column in that file.

**High-level methodology:**

1. **Represent the IBX alignment**
   - Load IBX station locations from `data/stations_ibx.geojson`.
   - Build an ordered **LineString** alignment joining the stations.
   - Keep the code modular so alternate station sets/alignments can be swapped in.

2. **Construct a 0.5-mile corridor (walkshed proxy)**
   - Project IBX alignment and stations into **EPSG:2263** (feet).
   - Buffer the IBX alignment by **0.5 miles (2,640 feet)** on both sides to approximate a half-mile corridor band.
   - Keep the buffer in EPSG:2263 for clean intersection with MapPLUTO, with an option to export to WGS84 for mapping.

3. **Load parcels (MapPLUTO) and link market valuations (DOF CSV)**
   - Load **MapPLUTO** parcel geometries from the FileGDB at `data/nyc_mappluto_25v3_fgdb` (layer `MAPPLUTO`).  
   - Load the **DOF assessment CSV** `data/DoF Valuation Data.csv`.
   - Harmonize parcel identifiers (BORO/BLOCK/LOT → BBL) and merge **`CURMKTLAND`** onto MapPLUTO parcels.
   - Use MapPLUTO’s **`AssessLand`** as the baseline **assessed land value**.

4. **Select eligible parcels within the 0.5-mile IBX corridor**
   - Spatially intersect parcels with the buffered alignment to define the **IBX corridor**.
   - Exclude parcels **not eligible** for value capture (city/state/federal/public authority; parks/open space; clearly non-tax parcels) using MapPLUTO’s `OwnerType` and `LandUse`.  
   - For each eligible parcel, retain:
     - **Current market land value**: `CURMKTLAND` (DOF CSV).
     - **Assessed land value**: `AssessLand` (MapPLUTO).

5. **Compute baseline totals and uplift scenarios**
   - Sum baseline **market land value** (`CURMKTLAND`) and **assessed land value** (`AssessLand`) in the IBX corridor.
   - Compute scenario land values under **4%, 6%, 8%, and 10%** uplift assumptions.
   - Visualize:
     - Parcels in/out of the corridor.
     - Corridor band + IBX line + stations.
     - Eligible vs ineligible parcels within the band.

6. **Gupta-style value capture estimation**
   - Assume that only a **fraction of the value uplift** is captured automatically via existing NYC property taxes (e.g., **30.6%** passive capture share, following Gupta et al.).
   - For each uplift scenario:
     - Compute the **incremental private land value** created in the IBX corridor.
     - Compute the **automatic property-tax capture** (baseline system) as that share of the uplift.
     - Given an assumed **IBX capital cost** and a **time horizon / discount rate**, solve for the **additional constant annual surcharge rate** on uplifted land value needed so that the present value of that surcharge equals the remaining unfunded cost.

7. **Final summary**
   - Print a compact summary including:
     - Baseline summed land market value (eligible parcels within IBX corridor).
     - Scenario uplifts (4/6/8/10%) and incremental value.
     - Assumed IBX capital cost, discount rate, horizon, and passive capture share.
     - Implied **annual surcharge rates** needed to close the funding gap under each scenario.

8. **Notebook export**
   - Provide a final cell that **exports this notebook** using `jupyter nbconvert` (e.g. to HTML) so it can be downloaded from a Jupyter environment.


## Setup and Configuration

Import libraries and define file paths, CRS, and key assumptions used throughout the pipeline.

In [ ]:
# This cell imports core libraries and sets global configuration for the analysis.
# Update paths and assumptions here if your environment differs.

import os
from pathlib import Path
from typing import Dict, Iterable
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt

load_dotenv()

# ------------------------------------------------------------------
# File paths (relative to project root)
# ------------------------------------------------------------------

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"


# IBX station locations (authoritative station geometry)
STATIONS_PATH = DATA_DIR / "stations_ibx.geojson"

# MapPLUTO: FileGDB folder and layer name
PLUTO_FGDB_PATH = DATA_DIR / "nyc_mappluto_25v3_fgdb/MapPLUTO25v3.gdb"
PLUTO_LAYER_NAME = (
    "MapPLUTO_25v3_clipped"  # adjust if the layer name differs in your FGDB
)

# DOF assessment CSV (valuation data, including CURMKTLAND)
VALUATION_CSV_PATH = DATA_DIR / "DoF Valuation Data.csv"

# ------------------------------------------------------------------
# CRS definitions
# ------------------------------------------------------------------
CRS_WGS84 = "EPSG:4326"  # geographic (lat/lon) for stations / maps
CRS_NY_FT = "EPSG:2263"  # NAD83 / New York Long Island (US feet); MapPLUTO native

# Half-mile buffer in feet
HALF_MILE_FEET = 0.5 * 5280  # 2,640 ft

# ------------------------------------------------------------------
# IBX alignment construction config
# ------------------------------------------------------------------
# Column used to sort stations along the IBX alignment. Must exist in stations_ibx.geojson.
STATION_SORT_COLUMN = "order"  # e.g., 'sequence', 'order', etc.

# ------------------------------------------------------------------
# Parcel / valuation schema configuration
# ------------------------------------------------------------------
# MapPLUTO parcel ID fields
PARCEL_ID_COLS = {
    "boro": "BoroCode",  # integer 1-5
    "block": "Block",  # integer
    "lot": "Lot",  # integer
    "bbl": "BBL",  # numeric BBL; we'll normalize to string
}

# DOF valuation CSV ID fields
VALUATION_ID_COLS = {
    "boro": "BORO",  # borough code
    "block": "BLOCK",  # block
    "lot": "LOT",  # lot
    "bbl": None,  # build a BBL string from BORO/BLOCK/LOT
}

# Land value fields
# - Market land value from DOF CSV.
# - Assessed land value from MapPLUTO (AssessLand), *not* from DOF, to avoid redundancy.
VALUATION_VALUE_FIELDS = {
    "land_market_value": "CURMKTLAND",  # Current Market Assessed Land Value (DOF CSV)
}
PLUTO_ASSESSED_LAND_COL = "AssessLand"  # MapPLUTO assessed land value

# MapPLUTO ownership / land-use fields
PLUTO_OWNER_COL = "OwnerType"
PLUTO_LANDUSE_COL = "LandUse"

# ------------------------------------------------------------------
# Gupta-style value capture assumptions
# ------------------------------------------------------------------
# Fraction of incremental market value captured (present value) by existing NYC tax system.
PASSIVE_CAPTURE_SHARE = 0.306

# IBX capital cost assumption (USD)
IBX_CAPITAL_COST = 5_500_000_000  # $5.5B in 2027 dollars

# Discount rate and horizon (for surcharge PV computation)
DISCOUNT_RATE = 0.03  # 3% real discount rate
HORIZON_YEARS = 30  # years

# Uplift rates to analyze
UPLIFT_RATES = [0.04, 0.06, 0.08, 0.10]

print("Configuration loaded. Make sure paths and column names match your local data.")

## Helper Functions

Utility helpers for BBL construction, CRS handling, and basic plotting.

In [ ]:
# This cell defines helper functions used across the pipeline.


def construct_bbl_from_parts(boro, block, lot) -> str:
    """Build BBL string from boro/block/lot."""

    def clean(val):
        s = str(val)
        s = s.replace(",", "")
        s = "".join(ch for ch in s if ch.isdigit())
        return int(s)

    b, blk, lt = (clean(x) for x in (boro, block, lot))
    return f"{b:1d}{blk:05d}{lt:04d}"


def add_bbl_column(
    df: pd.DataFrame, id_cols: Dict[str, str], bbl_col_name: str = "BBL"
) -> pd.DataFrame:
    """Ensure a BBL column exists, either by using an existing column or constructing from BORO/BLOCK/LOT.

    Parameters
    ----------
    df : DataFrame
        Input dataframe with parcel ID columns.
    id_cols : dict
        Mapping with keys 'boro', 'block', 'lot', 'bbl' giving column names in df.
    bbl_col_name : str
        Output column name for BBL.
    """
    df = df.copy()
    bbl_existing = id_cols.get("bbl")
    if bbl_existing and bbl_existing in df.columns:
        # Normalize numeric BBL types to zero-padded strings
        def _norm(v):
            if pd.isna(v):
                return np.nan
            s = str(int(v))
            return s.zfill(10)

        df[bbl_col_name] = df[bbl_existing].apply(_norm)
    else:
        missing = [
            k for k in ("boro", "block", "lot") if id_cols.get(k) not in df.columns
        ]
        if missing:
            raise KeyError(f"Cannot construct BBL; missing columns: {missing}")
        df[bbl_col_name] = [
            construct_bbl_from_parts(b, blk, lt)
            for b, blk, lt in zip(
                df[id_cols["boro"]], df[id_cols["block"]], df[id_cols["lot"]]
            )
        ]
    return df


def project_gdf(gdf: gpd.GeoDataFrame, to_crs: str) -> gpd.GeoDataFrame:
    """Reproject a GeoDataFrame to the given CRS if needed."""
    if gdf.crs is None:
        gdf = gdf.set_crs(to_crs)
    if gdf.crs.to_string() != to_crs:
        return gdf.to_crs(to_crs)
    return gdf


def quick_plot_base(parcels: gpd.GeoDataFrame, title: str = "", figsize=(8, 8)):
    """Quick base map of parcels for visual sanity checks."""
    ax = parcels.plot(edgecolor="none", alpha=0.3, figsize=figsize)
    ax.set_title(title)
    plt.axis("equal")
    return ax

## 1. Draw IBX Alignment as Interpretable Geometry

This step:

1. Loads IBX station locations from `data/stations_ibx.geojson`.
2. Sorts stations along the corridor using `STATION_SORT_COLUMN`.
3. Builds a **LineString** representing the IBX alignment.
4. Keeps everything modular so alternate station sets/alignments can be swapped in later.


In [ ]:
# This cell defines functions to load station points and construct the IBX alignment line.


def load_stations(
    stations_path: Path, sort_col: str = STATION_SORT_COLUMN
) -> gpd.GeoDataFrame:
    """Load IBX station points from stations_ibx.geojson and sort them along the corridor.

    The GeoJSON is assumed to contain:
      - Point geometries for each proposed IBX station.
      - A column (default: 'sequence') that orders stations along the line.
    """
    gdf = gpd.read_file(stations_path)
    if sort_col not in gdf.columns:
        raise KeyError(
            f"Station sort column '{sort_col}' not found in {stations_path}. Columns: {list(gdf.columns)}"
        )
    gdf = gdf.sort_values(sort_col).reset_index(drop=True)
    if gdf.crs is None:
        gdf = gdf.set_crs(CRS_WGS84)
    return gdf


def build_alignment_line(stations_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Construct a LineString alignment from ordered station points."""
    line = LineString(list(stations_gdf.geometry))
    alignment = gpd.GeoDataFrame(
        pd.DataFrame({"segment_id": ["IBX_main"]}),
        geometry=[line],
        crs=stations_gdf.crs,
    )
    return alignment


# Execute step 1
stations_gdf = load_stations(STATIONS_PATH, sort_col=STATION_SORT_COLUMN)
alignment_gdf = build_alignment_line(stations_gdf)

print(
    f"Loaded {len(stations_gdf)} stations from {STATIONS_PATH} and built IBX alignment line."
)
alignment_gdf.head()

In [ ]:
# Visual sanity check: plot stations and IBX alignment over a simple base map.

fig, ax = plt.subplots(figsize=(6, 6))
alignment_gdf.plot(ax=ax, color="black", linewidth=2, label="IBX Alignment")
stations_gdf.plot(ax=ax, color="red", markersize=30, label="Stations")
ax.set_title("IBX Stations and Alignment (from stations_ibx.geojson)")
ax.legend()
plt.axis("equal")
plt.show()

## 2. Construct 0.5-Mile IBX Corridor Buffer

Here we:

1. Project the IBX alignment from WGS84 into **EPSG:2263** (feet), matching MapPLUTO.
2. Buffer the alignment by **0.5 miles / 2,640 feet**.
3. Keep the buffer in EPSG:2263 for clean intersection with parcels, with an optional WGS84 export for mapping.

This yields a **corridor band** approximating a half-mile walkshed around the IBX right-of-way.


In [ ]:
# This cell buffers the IBX alignment by 0.5 miles in EPSG:2263.


def build_ibx_buffer(
    alignment_gdf: gpd.GeoDataFrame,
    buffer_distance_ft: float = HALF_MILE_FEET,
    to_crs: str = CRS_NY_FT,
) -> gpd.GeoDataFrame:
    """Buffer the IBX alignment by a given distance (in feet) and return a polygon GeoDataFrame in EPSG:2263."""
    aln_proj = alignment_gdf.to_crs(to_crs)
    buffered_geom = aln_proj.buffer(buffer_distance_ft)
    buffer_gdf = gpd.GeoDataFrame(
        aln_proj.drop(columns="geometry"), geometry=buffered_geom, crs=to_crs
    )
    buffer_gdf["buffer_ft"] = buffer_distance_ft
    return buffer_gdf


ibx_buffer_gdf = build_ibx_buffer(alignment_gdf, buffer_distance_ft=HALF_MILE_FEET)

print("IBX buffer geometry (EPSG:2263):")
ibx_buffer_gdf.head()

In [ ]:
# Visual sanity check: IBX buffer plus alignment (plotted in projected coordinates).

fig, ax = plt.subplots(figsize=(6, 6))
ibx_buffer_gdf.plot(
    ax=ax, color="lightblue", alpha=0.3, edgecolor="blue", label="0.5 mile buffer"
)
alignment_gdf.to_crs(CRS_NY_FT).plot(
    ax=ax, color="black", linewidth=2, label="IBX alignment"
)
ax.set_title("IBX 0.5-Mile Corridor Buffer (EPSG:2263)")
ax.legend()
plt.axis("equal")
plt.show()

## 3. Load Parcel Geometries (MapPLUTO) and Link DOF Market Land Values

This step:

1. Loads **MapPLUTO** parcel geometries from the file geodatabase at `data/nyc_mappluto_25v3_fgdb` (layer `MapPLUTO_25v3_clipped`), in **EPSG:2263**.  
2. Loads the **DOF assessment CSV** containing `CURMKTLAND` and other fields.
3. Ensures both datasets share a common **BBL identifier** (constructed from BORO/BLOCK/LOT where needed).  
4. Merges **current market land value** (`CURMKTLAND`) onto MapPLUTO parcels.
5. Uses MapPLUTO’s **`AssessLand`** as the parcel-level **assessed land value**, avoiding redundant DOF assessed fields.


In [ ]:
# This cell defines functions to load MapPLUTO and DOF CSV, harmonize IDs, and merge valuations.


def load_parcels_from_fgdb(
    fgdb_path: Path,
    layer_name: str = PLUTO_LAYER_NAME,
    parcel_id_cols: Dict[str, str] = PARCEL_ID_COLS,
) -> gpd.GeoDataFrame:
    """Load MapPLUTO parcels from a FileGDB and ensure a BBL column exists (string)."""
    gdf = gpd.read_file(fgdb_path, layer=layer_name)
    gdf = project_gdf(gdf, to_crs=CRS_NY_FT)
    gdf = add_bbl_column(gdf, parcel_id_cols, bbl_col_name="BBL")
    return gdf


def _clean_numeric(series: pd.Series) -> pd.Series:
    """Strip non-digits (e.g., commas) and return as string."""
    return series.astype(str).str.replace(r"\D", "", regex=True)


def load_valuation_csv(
    csv_path: Path,
    id_cols: Dict[str, str] = VALUATION_ID_COLS,
    value_fields: Dict[str, str] = VALUATION_VALUE_FIELDS,
) -> pd.DataFrame:
    df = pd.read_csv(csv_path, dtype=str, low_memory=False)

    # Clean IDs and build BBL
    for col in (id_cols["boro"], id_cols["block"], id_cols["lot"]):
        df[col] = df[col].astype(str).str.replace(r"\D", "", regex=True)
    df = add_bbl_column(df, id_cols, bbl_col_name="BBL")

    missing = [v for v in value_fields.values() if v not in df.columns]
    if missing:
        raise KeyError(f"Missing value columns: {missing}")

    # Clean and numeric value columns, then collapse duplicate BBLs
    for col in value_fields.values():
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[^0-9.\-]", "", regex=True)
            .replace("", pd.NA)
            .astype(float)
        )
    agg_spec = {col: "sum" for col in value_fields.values()}
    df = df.groupby("BBL", as_index=False).agg(agg_spec)
    return df


def merge_parcels_and_values(
    parcels_gdf: gpd.GeoDataFrame,
    values_df: pd.DataFrame,
    value_fields: Dict[str, str] = VALUATION_VALUE_FIELDS,
) -> gpd.GeoDataFrame:
    """Merge valuation data onto parcels via BBL and ensure numeric land value fields."""
    merged = parcels_gdf.merge(
        values_df[["BBL"] + list(value_fields.values())],
        on="BBL",
        how="left",
        validate="1:1",
    )
    for label, col in value_fields.items():
        merged[col] = pd.to_numeric(merged[col], errors="coerce")
    if PLUTO_ASSESSED_LAND_COL in merged.columns:
        merged[PLUTO_ASSESSED_LAND_COL] = pd.to_numeric(
            merged[PLUTO_ASSESSED_LAND_COL], errors="coerce"
        )
    else:
        raise KeyError(
            f"Expected MapPLUTO assessed land column '{PLUTO_ASSESSED_LAND_COL}' not found."
        )
    return merged


# Execute data loading
parcels_gdf = load_parcels_from_fgdb(PLUTO_FGDB_PATH, layer_name=PLUTO_LAYER_NAME)
print(f"Loaded {len(parcels_gdf):,} parcels from MapPLUTO. CRS: {parcels_gdf.crs}")

valuation_df = load_valuation_csv(VALUATION_CSV_PATH)
print(f"Loaded {len(valuation_df):,} valuation rows from DOF CSV (deduped by BBL).")

parcels_with_values_gdf = merge_parcels_and_values(parcels_gdf, valuation_df)
print(f"Merged parcels and valuation data. Rows: {len(parcels_with_values_gdf):,}")

parcels_with_values_gdf[
    ["BBL", "AssessLand", VALUATION_VALUE_FIELDS["land_market_value"]]
].head()


## 4. Select IBX-Proximate, Value-Capture-Eligible Parcels

We now:

1. Select MapPLUTO parcels whose geometry intersects the **0.5-mile IBX buffer** (all work in EPSG:2263).
2. Apply **eligibility filters** to remove parcels that are not plausible value-capture targets, using:
   - `OwnerType`: owner categories (e.g., private vs public).
   - `LandUse`: 11 land-use categories derived from DOF building classes.

Illustrative heuristic (tunable):

- **Eligible OwnerType**: P (private) and blank (unknown, usually private).  
- **Ineligible LandUse**: 07 (Transportation & Utility), 08 (Public Facilities & Institutions), 09 (Open Space & Outdoor Recreation), 10 (Parking Facilities), 11 (Vacant Land).  

The exact codes will depend on the legal definition of the value-capture base; this code keeps the filters centralized and easily editable.


In [ ]:
# This cell defines functions to perform the buffer intersection and eligibility filtering.

### REMEMBER TO EXCLUDE ALL INELIGIBLE PARCELS IN FINAL ANALYSIS ###

from shapely import union_all  # shapely>=2


def select_parcels_in_buffer(
    parcels_gdf: gpd.GeoDataFrame, buffer_gdf: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    """Return parcels whose geometry intersects the IBX buffer polygon (all in EPSG:2263)."""
    parcels_proj = project_gdf(parcels_gdf, to_crs=CRS_NY_FT)
    buffer_proj = project_gdf(buffer_gdf, to_crs=CRS_NY_FT)
    buffer_union = union_all(buffer_proj.geometry)
    mask = parcels_proj.intersects(buffer_union)
    return parcels_proj.loc[mask].copy()


def filter_eligible_parcels(
    parcels_gdf: gpd.GeoDataFrame,
    owner_col: str = PLUTO_OWNER_COL,
    landuse_col: str = PLUTO_LANDUSE_COL,
) -> gpd.GeoDataFrame:
    """Keep parcels that are likely taxable; drop public owners and public/park land uses."""
    gdf = parcels_gdf.copy()

    # Drop obviously public owners; keep everything else (including unknowns)
    if owner_col in gdf.columns:
        owner_series = gdf[owner_col].astype(str).str.upper().fillna("")
        public_owner_types = {
            "C",
            "S",
            "F",
            "A",
            "M",
        }  # City/State/Federal/Authority/Muni
        mask_owner = ~owner_series.isin(public_owner_types)
    else:
        mask_owner = np.ones(len(gdf), dtype=bool)

    # Drop public facilities and open space/parks; keep other uses
    if landuse_col in gdf.columns:
        landuse_series = gdf[landuse_col].astype(str).str.zfill(2)
        ineligible_landuse = {"07", "08"}  # 07 public facilities, 08 open space/parks
        mask_land = ~landuse_series.isin(ineligible_landuse)
    else:
        mask_land = np.ones(len(gdf), dtype=bool)

    return gdf.loc[mask_owner & mask_land].copy()


# Execute selection and filtering
parcels_ibx_corridor_gdf = select_parcels_in_buffer(
    parcels_with_values_gdf, ibx_buffer_gdf
)
print(f"Parcels intersecting IBX 0.5-mile corridor: {len(parcels_ibx_corridor_gdf):,}")

eligible_ibx_parcels_gdf = filter_eligible_parcels(parcels_ibx_corridor_gdf)
print(f"Eligible parcels within IBX corridor: {len(eligible_ibx_parcels_gdf):,}")

eligible_ibx_parcels_gdf[
    ["BBL", "AssessLand", VALUATION_VALUE_FIELDS["land_market_value"]]
].head()


In [ ]:
# Visual inspection: parcels within IBX corridor, highlighting eligible vs ineligible (projected CRS).

fig, ax = plt.subplots(figsize=(8, 8))

parcels_ibx_corridor_gdf.plot(
    ax=ax, color="lightgrey", linewidth=0, alpha=0.4, label="In-corridor parcels"
)
eligible_ibx_parcels_gdf.plot(
    ax=ax, color="orange", linewidth=0, alpha=0.7, label="Eligible parcels"
)

ibx_buffer_gdf.boundary.plot(
    ax=ax, color="blue", linewidth=1, label="0.5 mile buffer boundary"
)
alignment_gdf.to_crs(CRS_NY_FT).plot(
    ax=ax, color="black", linewidth=2, label="IBX alignment"
)
stations_gdf.to_crs(CRS_NY_FT).plot(ax=ax, color="red", markersize=15, label="Stations")

ax.set_title("Parcels within IBX 0.5-Mile Corridor (eligible vs ineligible)")
ax.legend()
plt.axis("equal")
plt.show()

## 5. Baseline Land Value and Uplift Scenarios

For parcels that are both:

- within the **0.5-mile IBX corridor**, and
- **eligible** for value capture,

we:

1. Sum baseline **market land value** (`CURMKTLAND`) and **assessed land value** (`AssessLand`).
2. Compute scenario land values under **4%, 6%, 8%, and 10%** uplift assumptions.
3. Store the results in a tidy DataFrame for later use in the value-capture calculations.


In [ ]:
# This cell computes baseline land values and simple uplift scenarios.

land_mv_col = VALUATION_VALUE_FIELDS["land_market_value"]
land_av_col = PLUTO_ASSESSED_LAND_COL


def compute_baseline_and_uplifts(
    parcels_gdf: gpd.GeoDataFrame,
    uplift_rates: Iterable[float] = UPLIFT_RATES,
    land_mv_col: str = land_mv_col,
    land_av_col: str = land_av_col,
) -> pd.DataFrame:
    """Compute baseline land value and uplift scenarios for eligible parcels."""
    baseline_land_mv = parcels_gdf[land_mv_col].sum(skipna=True)
    baseline_land_av = parcels_gdf[land_av_col].sum(skipna=True)

    records = []
    for r in uplift_rates:
        scenario_mv = baseline_land_mv * (1 + r)
        incremental = scenario_mv - baseline_land_mv
        records.append(
            {
                "uplift_rate": r,
                "baseline_land_market_value": baseline_land_mv,
                "baseline_land_assessed_value": baseline_land_av,
                "scenario_land_market_value": scenario_mv,
                "incremental_land_value": incremental,
            }
        )
    return pd.DataFrame(records)


uplift_summary_df = compute_baseline_and_uplifts(eligible_ibx_parcels_gdf)
uplift_summary_df

## 6. Gupta-Style Value Capture Tax-Rate Estimation

We adopt a simplified version of the Gupta–Van Nieuwerburgh–Kontokosta framework:

1. For each uplift scenario, let \( \Delta V \) be the **incremental market land value** in the IBX corridor.
2. Assume the existing tax system passively captures a fixed share \( \theta \) of that uplift in present value (default \( \theta = 0.306 \)).
3. Remaining unfunded cost: \( C_{gap} = \max(C - \theta \Delta V, 0) \), where \( C \) is IBX capital cost.
4. Assume a constant **annual surcharge rate** \( \tau \) on the uplifted land value (\( \Delta V \)) over \( T \) years at discount rate \( r \):
   - Annual surcharge revenue: \( \tau \Delta V \).
   - Present value: \( \tau \Delta V A(r, T) \), where \( A(r, T) = \frac{1 - (1+r)^{-T}}{r} \).
   - Solve for \( \tau \) such that \( \tau \Delta V A(r,T) = C_{gap} \):
     \[
     \tau = \frac{C_{gap}}{\Delta V A(r, T)}.
     \]

We report \( \tau \) as an **annual value-capture tax rate** on the incremental land value (e.g., 0.5% of uplift per year).


In [ ]:
# This cell defines value-capture functions using the simplified Gupta-style framework.


def annuity_factor(r: float, T: int) -> float:
    """Present-value factor of a $1-per-year annuity over T years at rate r."""
    if r <= 0:
        return float(T)
    return (1 - (1 + r) ** (-T)) / r


def compute_surcharge_rate_for_scenario(
    incremental_value: float,
    capital_cost: float = IBX_CAPITAL_COST,
    passive_capture_share: float = PASSIVE_CAPTURE_SHARE,
    discount_rate: float = DISCOUNT_RATE,
    horizon_years: int = HORIZON_YEARS,
):
    """Compute funding gap and implied annual surcharge rate on uplifted land value."""
    passive_capture_pv = passive_capture_share * incremental_value
    funding_gap = max(capital_cost - passive_capture_pv, 0)
    A = annuity_factor(discount_rate, horizon_years)

    if incremental_value <= 0 or A <= 0 or funding_gap == 0:
        surcharge_rate = 0.0
    else:
        surcharge_rate = funding_gap / (incremental_value * A)

    return {
        "incremental_value": incremental_value,
        "passive_capture_pv": passive_capture_pv,
        "funding_gap": funding_gap,
        "annuity_factor": A,
        "surcharge_rate": surcharge_rate,
    }


def attach_value_capture_results(
    uplift_df: pd.DataFrame,
    capital_cost: float = IBX_CAPITAL_COST,
    passive_capture_share: float = PASSIVE_CAPTURE_SHARE,
    discount_rate: float = DISCOUNT_RATE,
    horizon_years: int = HORIZON_YEARS,
) -> pd.DataFrame:
    """Augment the uplift summary with Gupta-style value capture metrics for each scenario."""
    records = []
    for _, row in uplift_df.iterrows():
        vc = compute_surcharge_rate_for_scenario(
            incremental_value=row["incremental_land_value"],
            capital_cost=capital_cost,
            passive_capture_share=passive_capture_share,
            discount_rate=discount_rate,
            horizon_years=horizon_years,
        )
        rec = row.to_dict()
        rec.update(vc)
        records.append(rec)
    return pd.DataFrame(records)


value_capture_df = attach_value_capture_results(uplift_summary_df)
value_capture_df[
    [
        "uplift_rate",
        "baseline_land_market_value",
        "scenario_land_market_value",
        "incremental_land_value",
        "passive_capture_pv",
        "funding_gap",
        "surcharge_rate",
    ]
]

In [ ]:
# Present surcharge rates in a more interpretable way (% of incremental land value per year).

summary_display = value_capture_df.copy()
summary_display["surcharge_rate_pct_per_year"] = 100 * summary_display["surcharge_rate"]
summary_display[
    [
        "uplift_rate",
        "incremental_land_value",
        "passive_capture_pv",
        "funding_gap",
        "surcharge_rate_pct_per_year",
    ]
]

## 7. Final Summary

This cell prints a concise summary of:

- Baseline summed **market land value** (`CURMKTLAND`) and **assessed land value** (`AssessLand`) in the IBX corridor (eligible parcels only).
- Scenario uplift values (4%, 6%, 8%, 10%).
- IBX capital cost and Gupta-style assumptions.
- Implied annual value-capture surcharge rates needed to close the funding gap under each scenario.


In [ ]:
# This cell prints a human-readable summary of key results.

baseline_land_mv = uplift_summary_df["baseline_land_market_value"].iloc[0]
baseline_land_av = uplift_summary_df["baseline_land_assessed_value"].iloc[0]

print("=== IBX Land Value and Value Capture Summary ===\n")
print(f"Eligible IBX-corridor parcels: {len(eligible_ibx_parcels_gdf):,}")
print(f"Baseline summed land MARKET value (CURMKTLAND):  ${baseline_land_mv:,.0f}")
print(f"Baseline summed land ASSESSED value (AssessLand): ${baseline_land_av:,.0f}\n")

print(f"IBX capital cost assumption:          ${IBX_CAPITAL_COST:,.0f}")
print(f"Passive capture share (Gupta-style):  {PASSIVE_CAPTURE_SHARE:.3f}")
print(f"Discount rate:                         {DISCOUNT_RATE:.3%}")
print(f"Horizon (years):                       {HORIZON_YEARS}\n")

for _, row in value_capture_df.iterrows():
    rate_pct = 100 * row["surcharge_rate"]
    print(
        f"Uplift scenario: {row['uplift_rate'] * 100:.0f}% uplift in land market value"
    )
    print(f"  Incremental land value:     ${row['incremental_land_value']:,.0f}")
    print(f"  Passive tax capture (PV):   ${row['passive_capture_pv']:,.0f}")
    print(f"  Remaining funding gap:      ${row['funding_gap']:,.0f}")
    print(
        f"  Required surcharge rate:    {rate_pct:.3f}% of incremental land value per year\n"
    )